#**Music Preferences and Local Events**#

#Presented by:#
**Aabaran Paudel**

**Ashok Timsina**

**Komal Neupane**




#Objective#
This project aims to provide personalized music recommendations and relevant local events based on a user's Spotify top tracks and geographic location. By leveraging the Spotify Web API and the Ticketmaster API, the project integrates user music preferences with nearby events to enhance the overall user experience.

#Components#
Spotify Top Tracks Analysis:

*Fetching Data:* Retrieve the user's top tracks from Spotify using the Spotify Web API. This includes track names, artists, and track IDs.
Genre Analysis: Utilize the spotipy library to analyze the genres associated with each track's artists. This step aggregates and identifies the genres that define the user's music taste.
User Location Detection:

*Event Retrieval:* Fetch local events from the Ticketmaster API based on the user's location. The events are filtered and displayed to show relevant concerts and music-related activities happening nearby.


#Workflow#
1) Retrieve User's Top Tracks:

Use Spotify Web API to get the top tracks for a user, including details about the tracks and associated artists.

2)Analyze Track Genres:

Extract genres from the tracks' artists using the spotipy library to understand the user's music preferences.

3)Find Local Events:

Query the Ticketmaster API to find and display local music events based on the location entered by the user.

This project aims to combine music preferences with local event information to provide a personalized experience, recommending nearby music events that align with the user’s taste in music.

In [11]:
import requests
from google.colab import userdata
from requests.auth import HTTPBasicAuth

 ***** Update this before running the code. *****

In [12]:
city = "Los Angeles"
state ="CA"
country = "US"

#**Fetching Spotify Top Tracks**#
We use the Spotify Web API to fetch a user's top tracks over the long term. We define a function fetch_web_api to handle HTTP requests and manage authorization with a bearer token. The get_top_tracks function requests the top 5 tracks from Spotify and processes the response to extract track names, artist names, and track IDs. This data is printed and stored for further analysis.

In [13]:
import requests

# Authorization token that must have been created previously
token = userdata.get('Spotify_token')
def fetch_web_api(endpoint, method, body=None):
    url = f'https://api.spotify.com/{endpoint}'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    response = requests.request(method, url, headers=headers, json=body)
    response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
    return response.json()

def get_top_tracks():
    # Endpoint reference: https://developer.spotify.com/documentation/web-api/reference/get-users-top-artists-and-tracks
    endpoint = 'v1/me/top/tracks?time_range=long_term&limit=5'
    response_data = fetch_web_api(endpoint, 'GET')
    return response_data['items']

top_tracks = get_top_tracks()
ids=[]
for track in top_tracks:
    name = track['name']
    artists = ', '.join(artist['name'] for artist in track['artists'])
    print(f'{name} by {artists}')
    id= track['id']
    ids.append(id)

Someone In The Crowd by Emma Stone, Callie Hernandez, Sonoya Mizuno, Jessica Rothe
Kaalpanik / Maayajastai by Bartika Eam Rai
Piano Man by Billy Joel
Najeek by Bartika Eam Rai
Chaudhvin Ka Chand Ho (From "Chaudhvin Ka Chand") by Mohammed Rafi


#**Analyzing Track Genres with Spotipy**#
We use the spotipy library to analyze genres of the top tracks retrieved from Spotify. After installing spotipy, we initialize it with client credentials and define a get_genres function. This function retrieves genre information for each track's artists by making API calls and aggregates all genres into a unique list. The resulting genres are printed out for further insights.

In [14]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 2.8 MB/s eta 0:00:00


In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
CLIENT_ID = userdata.get('spotipy_client_id')
CLIENT_SECRET = userdata.get('spotipy_client_secret')

def get_genres(track_id):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

    track_data = sp.track(track_id)

    artist_ids = []

    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])

    artists_data = sp.artists(artist_ids)

    genres = []

    for artist in artists_data["artists"]:
        genres += artist["genres"]

    genres = set(genres)  # Removes duplicates by using a set

    return genres

# List of track IDs (assuming ids is defined somewhere in your code)
genres_list = []
for id in ids:
    genres = get_genres(id)
    genres_list.append(list(genres))  # Convert set to list and append
flat_genres_list = list(set([genre for sublist in genres_list for genre in sublist]))
#print(flat_genres_list)


# breaking two or three word genre into one
preferred_genre = []
for genre in flat_genres_list:
  preferred_genre.append(genre.split(' '))

preferred_genre_ = list(set([genre for sublist in preferred_genre for genre in sublist]))
print(preferred_genre_)

['gold', 'nepali', 'indie', 'classic', 'piano', 'bollywood', 'filmi', 'album', 'hollywood', 'soft', 'rock', 'singer-songwriter', 'mellow']


#**Fetching Local Events from Ticketmaster**#
We define a function get_events to fetch local events using the Ticketmaster API. The function constructs an API request to retrieve events and their details, including date, time, venue, genre, and ticket status. The results are printed to provide an overview of upcoming events relevant to the user's location enter in the beginning.



In [16]:
import requests
from datetime import datetime, timedelta

# function to get events
def get_events(api_key, **kwargs):
    base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

    # parameters for api call
    params = {
        "apikey": api_key,
        "size": 200,
        "sort": "date,asc"
    }

    # Add any additional parameters passed to the function
    params.update(kwargs)

    # if startDateTime and endDateTime are not provided, set them to this week
    if "startDateTime" not in params and "endDateTime" not in params:
        today = datetime.now()
        start_of_week = today - timedelta(days=today.weekday())
        end_of_week = start_of_week + timedelta(days=6)
        params["startDateTime"] = start_of_week.strftime('%Y-%m-%dT%H:%M:%SZ')
        params["endDateTime"] = end_of_week.strftime('%Y-%m-%dT%H:%M:%SZ')

    # request response from the api
    response = requests.get(base_url, params=params)

    # 200 status code is for success
    if response.status_code == 200:
        data = response.json()
        return data.get("_embedded", {}).get("events", [])
    else:
        print(f"Error: {response.status_code}")
        return []


# function to print events to display to the user
def print_events(events, preferred_list):
    for event in events:
      if 'classifications' in event and event['classifications']:
            genre = event['classifications'][0].get('genre', {}).get('name', 'N/A') # getting genre
            if genre.lower()in preferred_list: # if genre is mached from the genre retrived from spotify, print

              # Printing the event information
              print(f"Event: {event['name']}")
              print(f"Date: {event['dates']['start'].get('localDate', 'N/A')}")
              print(f"Time: {event['dates']['start'].get('localTime', 'N/A')}")
              print(f"Venue: {event['_embedded']['venues'][0]['name']}")
              print(f"Genre: {genre}")
              print(f"Ticket Status: {event['dates']['status']['code']}")
              print(f"More Info: {event['url']}")
              print("----------------------------------------------------")

api_key = userdata.get('ticketmaster_api')  # API Key

# Example usage
events = get_events(api_key,
                    city = city,
                    stateCode= state,
                    countryCode= country,
                    classificationName=["music"],
                    includeFamily="yes")

# Displaying the events to user
if events:
    print(f"Found {len(events)} events:")
    print_events(events, preferred_genre_)
else:
    print("No events found.")

Found 51 events:
Event: Goldford
Date: 2024-09-19
Time: 18:30:00
Venue: The Moroccan Lounge
Genre: Rock
Ticket Status: onsale
More Info: https://www.ticketmaster.com/goldford-los-angeles-california-09-19-2024/event/090060D4E70E3EF2
----------------------------------------------------
Event: Everclear
Date: 2024-09-19
Time: 19:00:00
Venue: The Regent Theater
Genre: Rock
Ticket Status: onsale
More Info: https://www.ticketmaster.com/everclear-los-angeles-california-09-19-2024/event/0900609381361698
----------------------------------------------------
Event: The Babies
Date: 2024-09-19
Time: 19:00:00
Venue: Teragram Ballroom
Genre: Rock
Ticket Status: onsale
More Info: https://www.ticketmaster.com/the-babies-los-angeles-california-09-19-2024/event/09006093951523E2
----------------------------------------------------
Event: Friday Pilots Club - The Nowhere Tour
Date: 2024-09-19
Time: 19:00:00
Venue: Echoplex
Genre: Rock
Ticket Status: onsale
More Info: https://concerts.livenation.com/friday